In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df=pd.read_csv("D:/Machine Learning/Data Files/Data Files/4. ST Academy - SVM resource files/Movie_classification.csv",header=0)
df.head()

,Marketing expense,Production expense,Multiplex coverage,Budget,Movie_length,Lead_ Actor_Rating,Lead_Actress_rating,Director_rating,Producer_rating,Critic_rating,Trailer_views,3D_available,Time_taken,Twitter_hastags,Genre,Avg_age_actors,Num_multiplex,Collection,Start_Tech_Oscar
0,20.1264,59.62,0.462,36524.125,138.7,7.825,8.095,7.910,7.995,7.94,527367,YES,109.60,223.840,Thriller,23,494,48000,1
1,20.5462,69.14,0.531,35668.655,152.4,7.505,7.650,7.440,7.470,7.44,494055,NO,146.64,243.456,Drama,42,462,43200,0
2,20.5458,69.14,0.531,39912.675,134.6,7.485,7.570,7.495,7.515,7.44,547051,NO,147.88,2022.400,Comedy,38,458,69400,1
3,20.6474,59.36,0.542,38873.890,119.3,6.895,7.035,6.920,7.020,8.26,516279,YES,185.36,225.344,Drama,45,472,66800,1
4,21.3810,59.36,0.542,39701.585,127.7,6.920,7.070,6.815,7.070,8.26,531448,NO,176.48,225.792,Drama,55,395,72400,1


In [3]:
df['Time_taken'].fillna(value=df['Time_taken'].mean(),inplace=True) #inplace=True to make changes in the file
df=pd.get_dummies(df,columns=['3D_available','Genre'],drop_first=True)
x=df.loc[:,df.columns != "Start_Tech_Oscar"]
y=df["Start_Tech_Oscar"]

In [4]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)
x_train.shape

(404, 20)

In [5]:
#Standardizing Data
from sklearn.preprocessing import StandardScaler
sc=StandardScaler().fit(x_train)
x_train_std=sc.transform(x_train)
x_test_std=sc.transform(x_test) #No need to standardise y variable

In [6]:
from sklearn import svm

In [7]:
clf_svm_1=svm.SVC(kernel='linear',C=0.01)
clf_svm_1.fit(x_train_std,y_train)

SVC(C=0.01, kernel='linear')

In [8]:
y_train_pred = clf_svm_1.predict(x_train_std)
y_test_pred = clf_svm_1.predict(x_test_std)
y_test_pred

array([1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0], dtype=int64)

In [9]:
from sklearn.metrics import accuracy_score,confusion_matrix

In [10]:
confusion_matrix(y_test,y_test_pred)

array([[11, 33],
       [ 5, 53]], dtype=int64)

In [11]:
accuracy_score(y_test,y_test_pred)

0.6274509803921569

In [12]:
clf_svm_1.n_support_ #To find number of support vectors [First Class,Second Class,....]

array([186, 189])

In [13]:
#Grid Search-To find the best value of C
from sklearn.model_selection import GridSearchCV

In [14]:
params = {'C':(0.001,0.005,0.01,0.05,0.1,0.5,1,5,10,50)}
clf_svm_1=svm.SVC(kernel='poly')
svm_grid_lin = GridSearchCV(clf_svm_1,params,n_jobs=-1,cv=10,verbose=1,scoring='accuracy') #Verbose->To get msg
svm_grid_lin.fit(x_train_std,y_train)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


GridSearchCV(cv=10, estimator=SVC(kernel='poly'), n_jobs=-1,
             param_grid={'C': (0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10,
                               50)},
             scoring='accuracy', verbose=1)

In [15]:
svm_grid_lin.best_params_ #Best Parameter

{'C': 1}

In [16]:
linsvm_clf=svm_grid_lin.best_estimator_ #Best Model
accuracy_score(y_test,linsvm_clf.predict(x_test_std))

0.5980392156862745

In [17]:
#Polynomial Kernel
clf_svm_p3=svm.SVC(kernel='poly',degree=2,C=0.1) #Degree is the degree of the polynomial
clf_svm_p3.fit(x_train_std,y_train)

SVC(C=0.1, degree=2, kernel='poly')

In [18]:
y_train_pred=clf_svm_p3.predict(x_train_std)
y_test_pred=clf_svm_p3.predict(x_test_std)

In [19]:
accuracy_score(y_test,y_test_pred)

0.5588235294117647

In [20]:
clf_svm_p3.n_support_

array([185, 194])

In [21]:
#Radial Kernel
clf_svm_r=svm.SVC(kernel='rbf',gamma=0.5,C=10) #Degree is the degree of the polynomial
clf_svm_r.fit(x_train_std,y_train)

SVC(C=10, gamma=0.5)

In [22]:
y_train_pred=clf_svm_r.predict(x_train_std)
y_test_pred=clf_svm_r.predict(x_test_std)
accuracy_score(y_test,y_test_pred)

0.6176470588235294

In [23]:
clf_svm_p3.n_support_

array([185, 194])

In [24]:
#Radial Grid
params = {'C':(0.01,0.05,0.1,0.5,1,5,10,50),'gamma':(0.001,0.01,0.1,0.5,1)}

In [26]:
clf_svm_r=svm.SVC(kernel='rbf')
svm_grid_rad = GridSearchCV(clf_svm_r,params,n_jobs=-1,cv=3,verbose=1,scoring='accuracy')

In [27]:
svm_grid_rad.fit(x_train_std,y_train)
svm_grid_rad.best_params_
radsvm_clf=svm_grid_rad.best_estimator_
accuracy_score(y_test,radsvm_clf.predict(x_test_std))

Fitting 3 folds for each of 40 candidates, totalling 120 fits


0.6176470588235294